In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


## import

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout, Conv1D, MaxPooling1D, GlobalAveragePooling1D, LSTM, Activation, GRU, SimpleRNN
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
import keras.callbacks as kcallbacks
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

from tensorflow.keras.utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
#20 normal + 20 anomalous = test / con lai la train
#fwap + http = train
import datetime
# Get the current date and time
current_datetime = datetime.datetime.now()
# Format the date and time as a string
date_string = current_datetime.strftime("%Y-%m-%d")

# Path

In [3]:
DATASET_TRAINING = '/content/drive/MyDrive/Colab_Notebooks/detect_attack_by_reinforcement_learning/data/matrix5/normal/train.csv'
DATASET_TESTING_csic='/content/drive/MyDrive/Colab_Notebooks/detect_attack_by_reinforcement_learning/data/matrix5/normal/TEST_OK_csic2010.csv'
DATASET_TESTING_fwaf='/content/drive/MyDrive/Colab_Notebooks/detect_attack_by_reinforcement_learning/data/matrix5/normal/TEST_OK_fwaf.csv'
DATASET_TESTING_httpParams='/content/drive/MyDrive/Colab_Notebooks/detect_attack_by_reinforcement_learning/data/matrix5/normal/TEST_OK_httpParams.csv'
DATASET_TESTING_ecml='/content/drive/MyDrive/Colab_Notebooks/detect_attack_by_reinforcement_learning/data/matrix5/normal/TEST_OK_ECML.csv'

## minmax

## binary

In [ ]:
# DATASET_TRAINING = '/content/drive/MyDrive/UIT/HocKy8/KLTN/detect_attack_by_reinforcement_learning/data/matrix3/min-max/train_minmax.csv'
# # DATASET_TESTING='/content/drive/MyDrive/UIT/HocKy8/KLTN/detect_attack_by_reinforcement_learning/data/matrix3/min-max/test_csic2010-minmax.csv'
# # DATASET_TESTING='/content/drive/MyDrive/UIT/HocKy8/KLTN/detect_attack_by_reinforcement_learning/data/matrix3/min-max/test_fwaf-minmax.csv'
# DATASET_TESTING='/content/drive/MyDrive/UIT/HocKy8/KLTN/detect_attack_by_reinforcement_learning/data/matrix3/min-max/test_http_param-minmax.csv'

# load data

In [4]:
### Load data
df_train = pd.read_csv(DATASET_TRAINING)
# display(df_train)
# df_test = pd.read_csv(DATASET_TESTING, header=None, index_col=False)
df_train = df_train.astype(float)
df_train = df_train.sample(frac = 1)
x_train, y_train = df_train.iloc[:, :-1], df_train.iloc[:, [-1]]
x_train = x_train/255.0
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1)

y_train = y_train.astype(int)
y_val = y_val.astype(int)

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

input_shape = x_train.shape[1]
output_shape = y_train.shape[1]



In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(input_shape, output_shape)

(97034, 51)
(97034, 2)
(10782, 51)
(10782, 2)
51 2


## load test
x_test and y_test represent the test data inputs and labels

In [5]:
df_test_csic = pd.read_csv(DATASET_TESTING_csic)

df_test_csic = df_test_csic.astype(float)

df_test_csic = df_test_csic.sample(frac = 1)

x_test_csic, y_test_csic = df_test_csic.iloc[:, :-1], df_test_csic.iloc[:, [-1]]

x_test_csic = x_test_csic/255.0
y_test_csic = y_test_csic.astype(int)
y_test_csic = to_categorical(y_test_csic)

In [6]:
print(x_test_csic.shape)
print(y_test_csic.shape)
print(x_test_csic.shape[0])


(2642, 51)
(2642, 2)
2642


In [7]:
df_test_fwaf = pd.read_csv(DATASET_TESTING_fwaf)

df_test_fwaf = df_test_fwaf.astype(float)

df_test_fwaf = df_test_fwaf.sample(frac = 1)

x_test_fwaf, y_test_fwaf = df_test_fwaf.iloc[:, :-1], df_test_fwaf.iloc[:, [-1]]

x_test_fwaf = x_test_fwaf/255.0
y_test_fwaf = y_test_fwaf.astype(int)
y_test_fwaf = to_categorical(y_test_fwaf)

In [8]:
df_test_httpParams = pd.read_csv(DATASET_TESTING_httpParams)

df_test_httpParams = df_test_httpParams.astype(float)

df_test_httpParams = df_test_httpParams.sample(frac = 1)

x_test_httpParams, y_test_httpParams = df_test_httpParams.iloc[:, :-1], df_test_httpParams.iloc[:, [-1]]

x_test_httpParams = x_test_httpParams/255.0
y_test_httpParams = y_test_httpParams.astype(int)
y_test_httpParams = to_categorical(y_test_httpParams)

In [9]:
df_test_ecml = pd.read_csv(DATASET_TESTING_ecml)

df_test_ecml = df_test_ecml.astype(float)

df_test_ecml = df_test_ecml.sample(frac = 1)

x_test_ecml, y_test_ecml = df_test_ecml.iloc[:, :-1], df_test_ecml.iloc[:, [-1]]

x_test_ecml = x_test_ecml/255.0
y_test_ecml = y_test_ecml.astype(int)
y_test_ecml = to_categorical(y_test_ecml)

In [10]:
checkpoint_filepath = '/content/drive/MyDrive/UIT/HocKy8/KLTN/checkpoints/weights.{epoch:02d}-{val_loss:.2f}.h5'
model_checkpoint_callback = kcallbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

# Evaluate


In [11]:
def evalutate_metric(x_test, y_test):
  import numpy as np
  ### Evaluate model
  y_pred  = model.predict(x_test)

  y_pred = np.argmax(y_pred, axis=1)
  y_test = np.argmax(y_test, axis=1)

  accuracy = accuracy_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)

  print("----------------------------------------------")
  print("accuracy: %.6f" %accuracy)
  print("recall: %.6f" %recall)
  print("precision: %.6f" %precision)
  print("f1_score: %.6f" %f1)
  print("confusion matrix")
  print("==============================================")
  cm = metrics.confusion_matrix(y_test, y_pred)
  print(cm)
  print("==============================================")

  import time
  import numpy as np

  prediction_times = []
  x_test_array = x_test.values
  for i in range(100):
      start_time = time.time()  # Start time
      prediction = model.predict(np.expand_dims(x_test_array[i], axis=0), verbose =0)
      end_time = time.time()  # End time
      prediction_time = end_time - start_time  # Calculate the prediction time
      prediction_times.append(prediction_time)  # Store the prediction time in the list

  average_time = sum(prediction_times) / len(prediction_times)
  minimum_time = min(prediction_times)
  maximum_time = max(prediction_times)

  print("Average prediction time:", average_time)
  print("Minimum prediction time:", minimum_time)
  print("Maximum prediction time:", maximum_time)
  print("---!!----")



# CNN


## CNN 2

In [19]:
### CNN 2
model = Sequential()

model.add(Conv1D(64, 3, activation="relu", padding='same', input_shape=(input_shape, 1)))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Flatten())
model.add(Dense(32, activation="relu"))
# model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(output_shape, activation="softmax"))

# define optimizer and objective, compile cnn

#### Chú ý đoạn này
model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])
# model.compile(loss="categorical_crossentropy", optimizer="adam")
# model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=METRICS) # for metrics
#### Chú ý đoạn này

early_stop = kcallbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, mode='auto')
#fit model
history=model.fit(x_train, y_train, epochs=100, batch_size=64,  shuffle=True) #epochs=20,

# # loss: 0.2433 - accuracy: 0.8846 - 128
# # loss: 0.2384 - accuracy: 0.8884 - 64
# # loss: 0.2347 - accuracy: 0.8934 - 32

model.save(f"/content/drive/MyDrive/Colab_Notebooks/detect_attack_by_reinforcement_learning/process_Data/model_DL/CNN_{date_string}.h5")


Epoch 1/100
1517/1517 [==============================] - 7s 5ms/step - loss: 0.2136 - accuracy: 0.9139
Epoch 2/100
1517/1517 [==============================] - 7s 5ms/step - loss: 0.0765 - accuracy: 0.9753
Epoch 3/100
1517/1517 [==============================] - 7s 5ms/step - loss: 0.0632 - accuracy: 0.9786
Epoch 4/100
1517/1517 [==============================] - 6s 4ms/step - loss: 0.0568 - accuracy: 0.9802
Epoch 5/100
1517/1517 [==============================] - 7s 4ms/step - loss: 0.0506 - accuracy: 0.9820
Epoch 6/100
1517/1517 [==============================] - 6s 4ms/step - loss: 0.0455 - accuracy: 0.9839
Epoch 7/100
1517/1517 [==============================] - 7s 5ms/step - loss: 0.0416 - accuracy: 0.9857
Epoch 8/100
1517/1517 [==============================] - 6s 4ms/step - loss: 0.0382 - accuracy: 0.9868
Epoch 9/100
1517/1517 [==============================] - 7s 5ms/step - loss: 0.0366 - accuracy: 0.9876
Epoch 10/100
1517/1517 [==============================] - 6s 4ms/step - l

In [20]:
print("CSIC_2010")
evalutate_metric(x_test_csic, y_test_csic)

CSIC_2010
83/83 [==============================] - 0s 1ms/step
----------------------------------------------
accuracy: 0.999621
recall: 0.998236
precision: 1.000000
f1_score: 0.999117
confusion matrix
[[2075    0]
 [   1  566]]
Average prediction time: 0.04827067613601685
Minimum prediction time: 0.03549623489379883
Maximum prediction time: 0.08009576797485352
---!!----


In [21]:
print("FWAF")
evalutate_metric(x_test_fwaf, y_test_fwaf)

FWAF
471/471 [==============================] - 1s 1ms/step
----------------------------------------------
accuracy: 0.998206
recall: 0.996468
precision: 1.000000
f1_score: 0.998231
confusion matrix
[[7405    0]
 [  27 7618]]
Average prediction time: 0.0409282636642456
Minimum prediction time: 0.03489828109741211
Maximum prediction time: 0.05896401405334473
---!!----


In [22]:
print("HttpParams")
evalutate_metric(x_test_httpParams, y_test_httpParams)

HttpParams
305/305 [==============================] - 0s 1ms/step
----------------------------------------------
accuracy: 0.999795
recall: 0.999149
precision: 1.000000
f1_score: 0.999574
confusion matrix
[[7405    0]
 [   2 2349]]
Average prediction time: 0.04151597261428833
Minimum prediction time: 0.036231279373168945
Maximum prediction time: 0.05547690391540527
---!!----


In [23]:
print("ECML")
evalutate_metric(x_test_ecml, y_test_ecml)

ECML
326/326 [==============================] - 0s 1ms/step
----------------------------------------------
accuracy: 0.949419
recall: 0.982277
precision: 0.949744
f1_score: 0.965737
confusion matrix
[[2465  393]
 [ 134 7427]]
Average prediction time: 0.046051723957061766
Minimum prediction time: 0.03560805320739746
Maximum prediction time: 0.07539582252502441
---!!----


# LSTM

## LSTM 2

In [44]:
## LSTM 2
model = Sequential()
model.add(LSTM(4,input_shape=(input_shape, 1)))
model.add(Dropout(0.1))
model.add(Dense(output_shape))
model.add(Activation('softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy' ])
history=model.fit(x_train, y_train, epochs=100, batch_size=64, shuffle=True)

model.save(f"/content/drive/MyDrive/Colab_Notebooks/detect_attack_by_reinforcement_learning/process_Data/model_DL/LSTM_{date_string}.h5")

# loss: 0.4497 - accuracy: 0.7483

Epoch 1/100
1517/1517 [==============================] - 22s 13ms/step - loss: 0.5256 - accuracy: 0.7396
Epoch 2/100
1517/1517 [==============================] - 21s 14ms/step - loss: 0.3537 - accuracy: 0.8743
Epoch 3/100
1517/1517 [==============================] - 19s 12ms/step - loss: 0.3282 - accuracy: 0.8984
Epoch 4/100
1517/1517 [==============================] - 18s 12ms/step - loss: 0.3094 - accuracy: 0.9028
Epoch 5/100
1517/1517 [==============================] - 19s 13ms/step - loss: 0.3143 - accuracy: 0.9000
Epoch 6/100
1517/1517 [==============================] - 19s 13ms/step - loss: 0.3101 - accuracy: 0.9017
Epoch 7/100
1517/1517 [==============================] - 19s 13ms/step - loss: 0.3039 - accuracy: 0.9051
Epoch 8/100
1517/1517 [==============================] - 18s 12ms/step - loss: 0.3054 - accuracy: 0.9047
Epoch 9/100
1517/1517 [==============================] - 19s 12ms/step - loss: 0.3043 - accuracy: 0.9050
Epoch 10/100
1517/1517 [==============================]

In [45]:
# print("CSIC_2010")
evalutate_metric(x_test_csic, y_test_csic)

83/83 [==============================] - 0s 2ms/step
----------------------------------------------
accuracy: 0.967070
recall: 0.851852
precision: 0.993827
f1_score: 0.917379
confusion matrix
[[2072    3]
 [  84  483]]
Average prediction time: 0.04449566125869751
Minimum prediction time: 0.03616642951965332
Maximum prediction time: 0.31497693061828613
---!!----


In [49]:
print("FWAF")
evalutate_metric(x_test_fwaf, y_test_fwaf)

FWAF
471/471 [==============================] - 1s 3ms/step
----------------------------------------------
accuracy: 0.947641
recall: 0.923349
precision: 0.972180
f1_score: 0.947135
confusion matrix
[[7203  202]
 [ 586 7059]]
Average prediction time: 0.05717950582504273
Minimum prediction time: 0.039350271224975586
Maximum prediction time: 0.5136919021606445
---!!----


In [47]:
print("HttpParams")
evalutate_metric(x_test_httpParams, y_test_httpParams)

HttpParams
305/305 [==============================] - 1s 3ms/step
----------------------------------------------
accuracy: 0.978167
recall: 0.995321
precision: 0.920535
f1_score: 0.956468
confusion matrix
[[7203  202]
 [  11 2340]]
Average prediction time: 0.045644102096557615
Minimum prediction time: 0.03586554527282715
Maximum prediction time: 0.07336854934692383
---!!----


In [48]:
print("ECML")
evalutate_metric(x_test_ecml, y_test_ecml)

ECML
326/326 [==============================] - 1s 2ms/step
----------------------------------------------
accuracy: 0.931951
recall: 0.983997
precision: 0.926756
f1_score: 0.954519
confusion matrix
[[2270  588]
 [ 121 7440]]
Average prediction time: 0.042650766372680664
Minimum prediction time: 0.03592967987060547
Maximum prediction time: 0.06027841567993164
---!!----


# GRU

## GRU 2

In [29]:
### GRU 2
model = Sequential()
model.add(GRU(8,input_shape=(input_shape, 1), return_sequences=True))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(GRU(8,input_shape=(input_shape, 1), return_sequences=False))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(Dense(output_shape))
model.add(Activation('softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy' ])

history=model.fit(x_train, y_train, epochs=100, batch_size=64, shuffle=True)
# loss: 0.4454 - accuracy: 0.7774
model.save(f"/content/drive/MyDrive/Colab_Notebooks/detect_attack_by_reinforcement_learning/process_Data/model_DL/GRU_{date_string}.h5")

Epoch 1/100
1517/1517 [==============================] - 49s 30ms/step - loss: 0.4990 - accuracy: 0.7577
Epoch 2/100
1517/1517 [==============================] - 46s 30ms/step - loss: 0.3312 - accuracy: 0.8882
Epoch 3/100
1517/1517 [==============================] - 46s 30ms/step - loss: 0.3132 - accuracy: 0.8964
Epoch 4/100
1517/1517 [==============================] - 47s 31ms/step - loss: 0.3018 - accuracy: 0.9017
Epoch 5/100
1517/1517 [==============================] - 47s 31ms/step - loss: 0.2927 - accuracy: 0.9029
Epoch 6/100
1517/1517 [==============================] - 46s 30ms/step - loss: 0.2646 - accuracy: 0.9052
Epoch 7/100
1517/1517 [==============================] - 46s 31ms/step - loss: 0.2165 - accuracy: 0.9183
Epoch 8/100
1517/1517 [==============================] - 46s 30ms/step - loss: 0.1732 - accuracy: 0.9396
Epoch 9/100
1517/1517 [==============================] - 48s 31ms/step - loss: 0.1515 - accuracy: 0.9457
Epoch 10/100
1517/1517 [==============================]

In [30]:
print("CSIC_2010")
evalutate_metric(x_test_csic, y_test_csic)

CSIC_2010
83/83 [==============================] - 1s 5ms/step
----------------------------------------------
accuracy: 0.999621
recall: 1.000000
precision: 0.998239
f1_score: 0.999119
confusion matrix
[[2074    1]
 [   0  567]]
Average prediction time: 0.05403313159942627
Minimum prediction time: 0.0383448600769043
Maximum prediction time: 0.5293657779693604
---!!----


In [31]:
print("FWAF")
evalutate_metric(x_test_fwaf, y_test_fwaf)

FWAF
471/471 [==============================] - 2s 5ms/step
----------------------------------------------
accuracy: 0.987907
recall: 0.977240
precision: 0.998930
f1_score: 0.987966
confusion matrix
[[7397    8]
 [ 174 7471]]
Average prediction time: 0.05520077705383301
Minimum prediction time: 0.03908681869506836
Maximum prediction time: 0.10425567626953125
---!!----


In [32]:
print("HttpParams")
evalutate_metric(x_test_httpParams, y_test_httpParams)

HttpParams
305/305 [==============================] - 1s 4ms/step
----------------------------------------------
accuracy: 0.997847
recall: 0.994470
precision: 0.996590
f1_score: 0.995529
confusion matrix
[[7397    8]
 [  13 2338]]
Average prediction time: 0.043781981468200684
Minimum prediction time: 0.03818202018737793
Maximum prediction time: 0.057030439376831055
---!!----


In [33]:
print("ECML")
evalutate_metric(x_test_ecml, y_test_ecml)

ECML
326/326 [==============================] - 2s 5ms/step
----------------------------------------------
accuracy: 0.900854
recall: 0.996032
precision: 0.882470
f1_score: 0.935819
confusion matrix
[[1855 1003]
 [  30 7531]]
Average prediction time: 0.048911328315734866
Minimum prediction time: 0.037755489349365234
Maximum prediction time: 0.07680845260620117
---!!----


## SimpleRNN 2

In [34]:
### SimpleRNN 2
model = Sequential()
model.add(SimpleRNN(8,input_shape=(input_shape, 1), return_sequences=True))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(SimpleRNN(8,input_shape=(input_shape, 1), return_sequences=False))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(Dense(output_shape))
model.add(Activation('softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy' ])

history=model.fit(x_train, y_train,  epochs=100, batch_size=64, shuffle=True)
# loss: 0.4497 - accuracy: 0.7519
model.save(f"/content/drive/MyDrive/Colab_Notebooks/detect_attack_by_reinforcement_learning/process_Data/model_DL/SimpleRNN_{date_string}.h5")


Epoch 1/100
1517/1517 [==============================] - 20s 12ms/step - loss: 0.3770 - accuracy: 0.8479
Epoch 2/100
1517/1517 [==============================] - 21s 14ms/step - loss: 0.2719 - accuracy: 0.9058
Epoch 3/100
1517/1517 [==============================] - 19s 13ms/step - loss: 0.2473 - accuracy: 0.9165
Epoch 4/100
1517/1517 [==============================] - 19s 13ms/step - loss: 0.2221 - accuracy: 0.9254
Epoch 5/100
1517/1517 [==============================] - 18s 12ms/step - loss: 0.2047 - accuracy: 0.9317
Epoch 6/100
1517/1517 [==============================] - 20s 13ms/step - loss: 0.2413 - accuracy: 0.9094
Epoch 7/100
1517/1517 [==============================] - 19s 12ms/step - loss: 0.1727 - accuracy: 0.9389
Epoch 8/100
1517/1517 [==============================] - 20s 13ms/step - loss: 0.1604 - accuracy: 0.9416
Epoch 9/100
1517/1517 [==============================] - 19s 12ms/step - loss: 0.1526 - accuracy: 0.9444
Epoch 10/100
1517/1517 [==============================]

In [35]:
print("CSIC_2010")
evalutate_metric(x_test_csic, y_test_csic)

CSIC_2010
83/83 [==============================] - 0s 3ms/step
----------------------------------------------
accuracy: 0.968584
recall: 0.855379
precision: 0.997942
f1_score: 0.921178
confusion matrix
[[2074    1]
 [  82  485]]
Average prediction time: 0.044672081470489504
Minimum prediction time: 0.035828351974487305
Maximum prediction time: 0.20231199264526367
---!!----


In [36]:
print("FWAF")
evalutate_metric(x_test_fwaf, y_test_fwaf)

FWAF
471/471 [==============================] - 2s 3ms/step
----------------------------------------------
accuracy: 0.964784
recall: 0.940222
precision: 0.989946
f1_score: 0.964444
confusion matrix
[[7332   73]
 [ 457 7188]]
Average prediction time: 0.049879586696624754
Minimum prediction time: 0.03706002235412598
Maximum prediction time: 0.08019614219665527
---!!----


In [37]:
print("HttpParams")
evalutate_metric(x_test_httpParams, y_test_httpParams)

HttpParams
305/305 [==============================] - 1s 3ms/step
----------------------------------------------
accuracy: 0.990365
recall: 0.991068
precision: 0.969621
f1_score: 0.980227
confusion matrix
[[7332   73]
 [  21 2330]]
Average prediction time: 0.04240739107131958
Minimum prediction time: 0.036594390869140625
Maximum prediction time: 0.062159061431884766
---!!----


In [38]:
print("ECML")
evalutate_metric(x_test_ecml, y_test_ecml)

ECML
326/326 [==============================] - 1s 3ms/step
----------------------------------------------
accuracy: 0.889145
recall: 0.992197
precision: 0.872528
f1_score: 0.928523
confusion matrix
[[1762 1096]
 [  59 7502]]
Average prediction time: 0.046884803771972655
Minimum prediction time: 0.03680896759033203
Maximum prediction time: 0.07218384742736816
---!!----


# DNN

In [12]:
### DNN
model = Sequential()
model.add(Dense(32,input_dim=input_shape,activation='relu'))
model.add(Dropout(0.01))
model.add(Dense(output_shape))
model.add(Activation('softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy' ])

history=model.fit(x_train, y_train,  epochs=100, batch_size=64,  shuffle=True) #epochs=100, batch_size=32
model.save(f"/content/drive/MyDrive/Colab_Notebooks/detect_attack_by_reinforcement_learning/process_Data/model_DL/DNN_{date_string}.h5")

# loss: 0.2518 - accuracy: 0.8580

Epoch 1/100
1517/1517 [==============================] - 3s 2ms/step - loss: 0.3892 - accuracy: 0.8286
Epoch 2/100
1517/1517 [==============================] - 2s 2ms/step - loss: 0.1378 - accuracy: 0.9546
Epoch 3/100
1517/1517 [==============================] - 2s 2ms/step - loss: 0.0840 - accuracy: 0.9676
Epoch 4/100
1517/1517 [==============================] - 3s 2ms/step - loss: 0.0662 - accuracy: 0.9758
Epoch 5/100
1517/1517 [==============================] - 3s 2ms/step - loss: 0.0581 - accuracy: 0.9787
Epoch 6/100
1517/1517 [==============================] - 2s 2ms/step - loss: 0.0534 - accuracy: 0.9801
Epoch 7/100
1517/1517 [==============================] - 2s 2ms/step - loss: 0.0505 - accuracy: 0.9811
Epoch 8/100
1517/1517 [==============================] - 2s 2ms/step - loss: 0.0483 - accuracy: 0.9818
Epoch 9/100
1517/1517 [==============================] - 2s 2ms/step - loss: 0.0462 - accuracy: 0.9827
Epoch 10/100
1517/1517 [==============================] - 3s 2ms/step - l

# Evaluate on test data

In [13]:
print("CSIC_2010")
evalutate_metric(x_test_csic, y_test_csic)

CSIC_2010
83/83 [==============================] - 0s 1ms/step
----------------------------------------------
accuracy: 0.989023
recall: 0.950617
precision: 0.998148
f1_score: 0.973803
confusion matrix
[[2074    1]
 [  28  539]]
Average prediction time: 0.05895341634750366
Minimum prediction time: 0.04741477966308594
Maximum prediction time: 0.09238243103027344
---!!----


In [14]:
print("FWAF")
evalutate_metric(x_test_fwaf, y_test_fwaf)

FWAF
471/471 [==============================] - 1s 2ms/step
----------------------------------------------
accuracy: 0.993887
recall: 0.987966
precision: 1.000000
f1_score: 0.993947
confusion matrix
[[7405    0]
 [  92 7553]]
Average prediction time: 0.05820694208145141
Minimum prediction time: 0.04739236831665039
Maximum prediction time: 0.10200119018554688
---!!----


In [15]:
print("HttpParams")
evalutate_metric(x_test_httpParams, y_test_httpParams)

HttpParams
305/305 [==============================] - 0s 1ms/step
----------------------------------------------
accuracy: 0.999282
recall: 0.997023
precision: 1.000000
f1_score: 0.998509
confusion matrix
[[7405    0]
 [   7 2344]]
Average prediction time: 0.061723213195800784
Minimum prediction time: 0.047911643981933594
Maximum prediction time: 0.30728840827941895
---!!----


In [16]:
print("ECML")
evalutate_metric(x_test_ecml, y_test_ecml)

ECML
326/326 [==============================] - 1s 2ms/step
----------------------------------------------
accuracy: 0.928400
recall: 0.989023
precision: 0.918560
f1_score: 0.952490
confusion matrix
[[2195  663]
 [  83 7478]]
Average prediction time: 0.056621196269989016
Minimum prediction time: 0.046953439712524414
Maximum prediction time: 0.10497093200683594
---!!----
